In [12]:
import influxdb_client, os, time
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

token = "PPrBwtUvQCWRGBzfQa3TYgcuuFA9EYHJ7r3OKg4VhU9D1zM79oV07rBKxa_Hd-74TOzHKCTZK4S9SAypvo_VyA=="
org = "artica"
url = "https://eu-central-1-1.aws.cloud2.influxdata.com"
host = "eu-central-1-1.aws.cloud2.influxdata.com"

write_client = influxdb_client.InfluxDBClient(url=url, token=token, org=org)
write_api = write_client.write_api(write_options=SYNCHRONOUS)
bucket="site-data"

In [6]:
# Define the write api

data = {
  "point1": {
    "location": "Klamath",
    "species": "bees",
    "count": 23,
  },
  "point2": {
    "location": "Portland",
    "species": "ants",
    "count": 30,
  },
  "point3": {
    "location": "Klamath",
    "species": "bees",
    "count": 28,
  },
  "point4": {
    "location": "Portland",
    "species": "ants",
    "count": 32,
  },
  "point5": {
    "location": "Klamath",
    "species": "bees",
    "count": 29,
  },
  "point6": {
    "location": "Portland",
    "species": "ants",
    "count": 40,
  },
}

for key in data:
  point = (
    Point("census")
    .tag("location", data[key]["location"])
    .field(data[key]["species"], data[key]["count"])
  )
  write_api.write(bucket=bucket, org=org, record=point)
  time.sleep(1) # separate points by 1 second

print("Complete. Return to the InfluxDB UI.")

Complete. Return to the InfluxDB UI.


In [16]:
from flightsql import FlightSQLClient

query = """SELECT * FROM 'census'"""

# Define the query client
query_client = FlightSQLClient(
    host = host,
    token = token,
    metadata={"bucket-name": "site-data"}
)

# Execute the query
info = query_client.execute(query)
reader = query_client.do_get(info.endpoints[0].ticket)

# Convert to dataframe
data = reader.read_all()
df = data.to_pandas().sort_values(by="time")
df

ArrowInvalid: Flight returned invalid argument error, with message: Error while planning query: Error during planning: table 'public.iox.census' not found